In [201]:
import numpy as np
import scipy.integrate as integrate
import pandas as pd

import datetime

from astroquery.jplhorizons import Horizons

import plotly.graph_objects as go

In [348]:
## Useful constants
G = 6.67428e-11
m_earth = 5.97219e24
m_sun = 1.9891e30
au = 149597870700 # 1 Astronomical unit in meters: https://cneos.jpl.nasa.gov/glossary/au.html
seconds_in_year = 1*365.25*24*60*60

# Table with parameters for each body
body_names  = ["Sun", "Earth", "Mars"]
body_ids    = [10, 399, 499]
body_colors = ["goldenrod", "green", "red"]
body_masses = [ # Manually retrieved from Horizons
    1.9891e30,
    5.97219e24,
    6.4171e23,
]
df_bodies = pd.DataFrame({
    "name": body_names,
    "mass": body_masses,
    "NAIF_ID": body_ids,
    "color": body_colors
})

# TODOs:
#### Core simulation:
- [x] 3D
- [ ] Add 2nd planet
- [ ] Add all planets
- [ ] Add asteroid
#### Validation:
- [x] Pull data from horizons
- [ ] Create an asteroid based on a past horizons timestamp/frame
- [ ] Create metrics to compare a single asteroid prediction

In [349]:
## Pull data from Horizons

# Enter the desired simulation length
start_date = "2023-04-10"
sim_days = 361
sim_steps = int(sim_days/2)

# Create stop time string based on desired simulation length
datetime_format = "%Y-%m-%d"
start_datetime = datetime.datetime.strptime(start_date, datetime_format)
stop_datetime = start_datetime + datetime.timedelta(days=sim_days)
stop_date = stop_datetime.strftime(datetime_format)

def query_body_data(id):
    query = Horizons(id=id, location='500@0', epochs={"start": start_date, "stop": stop_date, "step": str(sim_steps - 1)})
    df = query.vectors().to_pandas()
    hrzns_pos = df[["x", "y", "z"]].to_numpy() * au    # Convert au to m
    hrzns_vel = df[["vx", "vy", "vz"]].to_numpy() * au / (24 * 60 * 60) # Convert au/d to m/s
    
    return hrzns_pos, hrzns_vel

bodies_pos = np.zeros((len(df_bodies.index), sim_steps, 3))
bodies_vel = np.zeros((len(df_bodies.index), sim_steps, 3))
for i, NAIF_ID in enumerate(df_bodies["NAIF_ID"]):
    pos, vel = query_body_data(NAIF_ID)
    bodies_pos[i, :, :] = pos
    bodies_vel[i, :, :] = vel

In [350]:
## Two-body problem
sim_earth_pos_0 = bodies_pos[1, 0, :]
sim_earth_vel_0 = bodies_vel[1, 0, :]

sim_mars_pos_0 = bodies_pos[2, 0, :]
sim_mars_vel_0 = bodies_vel[2, 0, :]

y_0 = np.concatenate([sim_earth_pos_0, sim_earth_vel_0])

n_bodies = len(df_bodies.index)
masses = df_bodies["mass"].to_numpy()

pos_0 = np.array([])
vel_0 = np.array([])
for i in df_bodies.index:
    pos_0 = np.append(pos_0, bodies_pos[i, 0, :])
    vel_0 = np.append(vel_0, bodies_vel[i, 0, :])

y_0 = np.concatenate([pos_0, vel_0])
    
def sun_earth_diffeq(t, y):
    dy = np.zeros(y.shape)
    
#     earth_to_sun = -y[0:3]
#     r_earth_to_sun = np.linalg.norm(earth_to_sun)
#     earth_to_sun = earth_to_sun / r_earth_to_sun
    
#     force_sun_to_earth = G * m_sun * m_earth / (r_earth_to_sun**2)
    
#     dy[0:3] = y[3:6]
#     dy[3:6] = earth_to_sun * force_sun_to_earth / m_earth
    
    dy[0:n_bodies * 3] = y[3*n_bodies:6*n_bodies]
    
    for i in df_bodies.index:
        pos_i = np.atleast_2d(y[3*i: 3*i + 3]).T
        pos_all = np.reshape(y[0:n_bodies*3], (n_bodies, 3)).T    # 3xn matrix where each column is the position of a body
        disp_all = pos_all - pos_i
        
        r_all = np.linalg.norm(disp_all, axis=0)
        accel_from_each = G * masses * r_all**-3 * disp_all
        accel_from_each = np.nan_to_num(accel_from_each)
        accel_i = np.sum(accel_from_each, axis=1)
        
        dy[3 * (n_bodies + i): 3 * (n_bodies + i + 1)] = accel_i
    
    return dy

n_days = sim_days
tspan = [0, n_days * 24 * 60 * 60]
n_points = sim_steps
t_eval = np.linspace(tspan[0], tspan[1], n_points)

soln_earth_sun = integrate.solve_ivp(sun_earth_diffeq, tspan, y_0, t_eval=t_eval)

/tmp/ipykernel_93530/304634652.py:41: RuntimeWarning:

divide by zero encountered in power

/tmp/ipykernel_93530/304634652.py:41: RuntimeWarning:

invalid value encountered in multiply



## 3D Results Plot

In [353]:
fig = go.Figure()

for i in df_bodies.index:
    body = df_bodies.iloc[i]
    
    fig.add_trace(go.Scatter3d(
        x=soln_earth_sun.y[3*i, :],
        y=soln_earth_sun.y[3*i+1, :],
        z=soln_earth_sun.y[3*i+2, :],
        name=f"Sim - {body['name']}",
        mode="lines+markers",
        marker={"size": 3, "color": "dark" + body["color"]}
    ))

# Plot horizons ground truth trajectories here:
for i in df_bodies.index:
    body = df_bodies.iloc[i]
    
    fig.add_trace(go.Scatter3d(
        x=bodies_pos[i, :, 0],
        y=bodies_pos[i, :, 1],
        z=bodies_pos[i, :, 2],
        name=f"Hrzn - {body['name']}",
        mode="lines+markers",
        marker={"size": 2, "color": body["color"], "symbol":"x"}
    ))

fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1), width=1200, height=800)
fig.show()